## Face mask detection with **Mobilenet Single-shot detector model**. 😷 🕵

##### This notebook demonstrates the procedure of training an object detection model for detecting face masks on peoples' faces.

##### The dataset used in this notebook has been downloadd from Kaggle. It contains three classes of images:
1. Class of images 'with mask'.
2. Class of images 'without mask'.
3. Class of images with 'mask weared incorrect'.

In [1]:
import os
import pathlib

#Clone tensorflow models repo if it doesn't exist.
if 'models' in pathlib.Path.cwd().parts:
  while 'models' in pathlib.Path.cwd().parts:
    os.chdir('..')

elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models


Cloning into 'models'...
remote: Enumerating objects: 3398, done.
remote: Counting objects: 100% (3398/3398), done.
remote: Compressing objects: 100% (2821/2821), done.
remote: Total 3398 (delta 896), reused 1410 (delta 520), pack-reused 0
Receiving objects: 100% (3398/3398), 34.94 MiB | 28.53 MiB/s, done.
Resolving deltas: 100% (896/896), done.


In [2]:
#Install the object-detection API.
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1694695 sha256=6b0df20904d76e32df1a473652647e78859cb7e5c0a290433e79a642f5654432
  Stored in directory: /tmp/pip-ephem-wheel-cache-5jhdmhlv/wheels/fa/a4/d2/e9a5057e414fd46c8e543d2706cd836d64e1fcd9eccceb2329
  Created wheel for py-cpuinfo: filename=py_cpuinfo-8.0.0-py3-none-any.whl size=22257 sha256=16def839e060bcae9d235ca5ef4f6b4b92b2cc7460c150c301115d06553443ee
  Stored in directory: /root/.cache/pip/wheels/d2/f1/1f/041add21dc9c4220157f1bd2bd6afe1f1a49524c3396b94401
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78544 sha256=c799e8682f73b962795b01c65f3acd67323a23cd2132a99f6de4e94411e50b30
  Stored in directory: /root/.cache/pip/wheels/a4/61/fd/c57e374e580aa78a45ed78d5859b3a44436af17e22ca53284f
  Created wheel for avro-python3: filename=avro_p

  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.13 requires dill>=0.3.5.1, but you have dill 0.3.1.1 which is incompatible.
gym 0.17.3 requires cloudpickle<1.7.0,>=1.2.0, but you have cloudpickle 2.1.0 which is incompatible.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.28.0 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
albumentations 0.1.12 requires imgaug<0.2.7,

In [3]:
import matplotlib
import matplotlib.pyplot as plt

import os
import io
import glob
import zipfile
import random 
import imageio
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf
from object_detection.utils import (label_map_util, config_util, colab_utils)
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [4]:
#Install opencv version 4.1.2.30

!pip install opencv-python-headless==4.1.2.30

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 21.8 MB 56.7 MB/s 
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.6.0.66
    Uninstalling opencv-python-headless-4.6.0.66:
      Successfully uninstalled opencv-python-headless-4.6.0.66


In [5]:
#Run model builder test.
!python '/content/models/research/object_detection/builders/model_builder_tf2_test.py'

2022-06-26 18:20:20.595003: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Running tests under Python 3.7.13: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
W0626 18:20:20.881385 140212399425408 model_builder.py:1102] Building experimental DeepMAC meta-arch. Some features may be omitted.
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 0.73s
I0626 18:20:21.327839 140212399425408 test_util.py:2459] time(__main__.ModelBuilderTF2Test.test_create_center_net_deepmac): 0.73s
[       OK ] ModelBuilderTF2Test.test_create_center_net_deepmac
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_center_net_model0 (customize_head_params=True)): 0.59s
I0626 18:20:21.923427 140212399425408 test_util.py:2459] time(__main__.ModelBuilderTF2Test.test_cr

In [6]:
def load_image_to_numpy_array(path):
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

#Plot detections functtion.
def plot_detections(image_np, boxes, classes, scores, category_index, figsize = (12, 16), image_name = None):
  '''Wrapper function to visualize detections.'''
  image_np_with_annotations = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_annotations, boxes, classes, scores,
      category_index, use_normalized_coordinates = True, 
      min_score_thresh = .8
  )
  if image_name:
    plt.imsave(image_name. image_np_with_annotations)
  else:
    plt.imshow(image_np_with_annotations)


## 2. Download and prepare data.

In [7]:
#Download zip file from gcs bucket.
!wget 'https://storage.googleapis.com/kimata/object_detection/02_face_mask_detection_mobilenetssd/inputs/face_mask_OD_kaggle.zip'

--2022-06-26 18:21:08--  https://storage.googleapis.com/kimata/object_detection/02_face_mask_detection_mobilenetssd/inputs/face_mask_OD_kaggle.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.216.128, 108.177.11.128, 173.194.218.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.216.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 416968083 (398M) [application/zip]
Saving to: ‘face_mask_OD_kaggle.zip’

face_mask_OD_kaggle 100%[===================>] 397.65M   236MB/s    in 1.7s    

2022-06-26 18:21:10 (236 MB/s) - ‘face_mask_OD_kaggle.zip’ saved [416968083/416968083]



In [8]:
#Extract zip file.
zip_ref = zipfile.ZipFile('/content/face_mask_OD_kaggle.zip', 'r')

if not os.path.exists('/content/inputs'):
  os.mkdir('/content/inputs')


zip_ref.extractall('/content/inputs')
zip_ref.close()
print('Done extracting files.')

Done extracting files.


In [9]:
#Create pbtxt file.
labels = [{'name': 'with_mask', 'id': 1}, {'name':'without_mask', 'id':2}, {'name': 'mask_weared_incorrect', 'id': 3}]

if not os.path.exists('/content/inputs/labelmap'):
  os.mkdir('/content/inputs/labelmap')

with open('/content/inputs/labelmap/label_map.pbtxt', 'w') as f:
    for label in labels:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(label['name']))
        f.write('\tid:{}\n'.format(label['id']))
        f.write('}\n')

In [12]:
#Clone the tfrecord generator module.
!git clone https://github.com/nicknochnack/GenerateTFRecord

#Check if tfrecord directory exists.
if not os.path.exists('/content/inputs/tfrecord_files'):
  os.mkdir('/content/inputs/tfrecord_files')

#Generate tfrecord files for training and text files.
!python '/content/GenerateTFRecord/generate_tfrecord.py' -i{'/content/inputs/images'} -x{'/content/inputs/annotations'} -l{'/content/inputs/labelmap/label_map.pbtxt'}  -o {'/content/inputs/tfrecord_files/dataset.tfrecord'}

fatal: destination path 'GenerateTFRecord' already exists and is not an empty directory.
2022-06-26 18:23:57.507147: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Traceback (most recent call last):
  File "/content/GenerateTFRecord/generate_tfrecord.py", line 168, in <module>
    tf.app.run()
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/platform/app.py", line 36, in run
    _run(main=main, argv=argv, flags_parser=_parse_flags_tolerate_undef)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 312, in run
    _run_main(main, args)
  File "/usr/local/lib/python3.7/dist-packages/absl/app.py", line 258, in _run_main
    sys.exit(main(argv))
  File "/content/GenerateTFRecord/generate_tfrecord.py", line 155, in main
    examples = xml_to_csv(args.xml_dir)
  File "/content/GenerateTFRecord/generate_tfrecord.py", line 88, in xml_to_csv
    int(member[4][0].text),
IndexError

In [13]:
#The above error is solved by changing lines 88 through 91 in the the generate_tfrecord.py file:
'''
88.    int(member[4][0].text),
89.    int(member[4][1].text),
90.    int(member[4][2].text),
91.     int(member[4][3].text)
)
 '''
#This is because different xml files store bounding box information in different indices. 
#We need to change the first index in the 'member' variable to 5 since bounding box coordinates are stored in index 5 in the xml files.


!python '/content/GenerateTFRecord/generate_tfrecord.py' -i{'/content/inputs/images'} -x{'/content/inputs/annotations'} -l{'/content/inputs/labelmap/label_map.pbtxt'}  -o {'/content/inputs/tfrecord_files/dataset.tfrecord'}

2022-06-26 18:29:26.515271: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Successfully created the TFRecord file: /content/inputs/tfrecord_files/dataset.tfrecord


In [14]:
train_record_fname = '/content/inputs/tfrecord_files/dataset.tfrecord'
test_record_fname = '/content/inputs/tfrecord_files/dataset.tfrecord'
label_map_pbtxt_fname = '/content/inputs/labelmap/label_map.pbtxt'

## Prepare, configure and train model.

In [15]:
MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_320x320_coco17_tpu-8',
        'base_pipeline_file': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.config',
        'pretrained_checkpoint': 'ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz',
        'batch_size': 32
    }
}

chosen_model = 'ssd_mobilenet_v2'
num_steps = 20000 #The more the steps, the longer the training.
num_eval_steps = 500 #Number of evaluation steps.
model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']
batch_size = MODELS_CONFIG[chosen_model]['batch_size']

In [16]:
#Download pretrained weights.
%mkdir '/content/models/research/deploy/'
%cd '/content/models/research/deploy/'

import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint
!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

/content/models/research/deploy
--2022-06-26 18:30:16--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.98.128, 2607:f8b0:400c:c1a::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.98.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46042990 (44M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz’

ssd_mobilenet_v2_32 100%[===================>]  43.91M   199MB/s    in 0.2s    

2022-06-26 18:30:17 (199 MB/s) - ‘ssd_mobilenet_v2_320x320_coco17_tpu-8.tar.gz’ saved [46042990/46042990]



In [17]:
base_pipeline_file

'ssd_mobilenet_v2_320x320_coco17_tpu-8.config'

In [18]:
#Download base config file.
%cd /content/models/research/deploy/
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
!wget {download_config}

/content/models/research/deploy
--2022-06-26 18:30:26--  https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_mobilenet_v2_320x320_coco17_tpu-8.config
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4484 (4.4K) [text/plain]
Saving to: ‘ssd_mobilenet_v2_320x320_coco17_tpu-8.config’

ssd_mobilenet_v2_32 100%[===================>]   4.38K  --.-KB/s    in 0s      

2022-06-26 18:30:26 (60.3 MB/s) - ‘ssd_mobilenet_v2_320x320_coco17_tpu-8.config’ saved [4484/4484]



In [19]:
#Prepare pipeline and fine-tune checkpoint.
pipeline_fname = '/content/models/research/deploy/ssd_mobilenet_v2_320x320_coco17_tpu-8.config'
fine_tune_checkpoint = '/content/models/research/deploy/' + model_name + '/checkpoint/ckpt-0'

def get_num_classes(pbtxt_fname):
  label_map = label_map_util.load_labelmap(pbtxt_fname)
  categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes = 90, use_display_name = True)
  category_index = label_map_util.create_category_index(categories)
  return len(category_index.keys())


num_classes = get_num_classes(label_map_pbtxt_fname)
num_classes

3

In [20]:
#write custom configuration file by slotting our dataset, model checkpoint, and training parameters into the base pipeline file

import re

%cd '/content/models/research/deploy'
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()
with open('pipeline_file.config', 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    #fine-tune checkpoint type
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
        
    f.write(s)

/content/models/research/deploy
writing custom configuration file


In [ ]:
## Train model.
!mkdir '/content/train'

pipeline_file = '/content/models/research/deploy/pipeline_file.config'
model_dir = '/content/train'


!python /content/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}

2022-05-17 12:56:09.549024: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0517 12:56:09.554370 139720908851072 mirrored_strategy.py:374] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 20000
I0517 12:56:09.558158 139720908851072 config_util.py:552] Maybe overwriting train_steps: 20000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0517 12:56:09.558314 139720908851072 config_util.py:552] Maybe overwriting use_bfloat16: False
Instructions for updating:
rename to distribute_datasets_from_function
W0517 12:56:09.707922 139720908851072 deprecation.py:343] From /usr/local/lib/python3.7/dist-packages/object_detection/model_lib_v2.py:564: StrategyBase.experime